In [92]:
import numpy as np
import pandas as pd

**Будем оптимизировать функцию MSE для линейной регрессии. MSE в матричном виде**


In [82]:
def gradient_descent(X: np.array, y : np.array, w0: np.array, learning_rate: float, iterations: int, eps=1e-4):

    '''Функция поиска минимума функции MSE в векторном виде с использованием классического градиентного спуска'''
    m = X.shape[0] # кол-во объектов
    if w0.shape[-1] != X.shape[-1] + 1:
        return 'Неверно задана матрица весов w0. Проверьте количество признаков'
    X = np.hstack((np.ones((m, 1)), X)) # добавляем константу для w0
    params = w0
    
    for _ in range(iterations):
        grad = X.T @ ((X @ params) - y) # X.T @ - вычисляет градиенты, ((X @ params) - y) - вычисляет ошибки по каждому объекту
        update =  (2./m) * learning_rate * grad

        if np.linalg.norm(update) < eps:
            return params
        params -= update

    return params

In [ ]:
def mini_batch(X: np.array, y : np.array, w0: np.array, learning_rate: float, iterations: int, batch=1, eps=1e-4):

    '''Функция поиска минимума функции MSE в векторном виде с использованием модификации mini batch. 
        По умолчанию параметр batch = 1, то есть используется Стохастический градиентный спуск.'''
    
    m = X.shape[0] # кол-во объектов
    if w0.shape[-1] != X.shape[-1] + 1:
        return 'Неверно задана матрица весов w0. Проверьте количество признаков'
    X = np.hstack((np.ones((m, 1)), X)) # добавляем константу для w0
    params = w0
    
    for _ in range(iterations):
        idxs = np.random.randint(0, m, batch)
        X_mini, y_mini = X[idxs], y[idxs]
        grad = X_mini.T @ ((X_mini @ params) - y_mini) # X.T @ - вычисляет градиенты, ((X @ params) - y) - вычисляет ошибки по каждому объекту
        update =  (2./batch) * learning_rate * grad

        if np.linalg.norm(update) < eps:
            return params
        params -= update

    return params

In [94]:
def momentum(X: np.array, y : np.array, w0: np.array, learning_rate: float, iterations: int, beta: int, Nesterov=False, eps=1e-4):

    '''Функция поиска минимума функции MSE в векторном виде с использованием метода моментов'''
    m = X.shape[0] # кол-во объектов
    if w0.shape[-1] != X.shape[-1] + 1:
        return 'Неверно задана матрица весов w0. Проверьте количество признаков'
    X = np.hstack((np.ones((m, 1)), X)) # добавляем константу для w0
    params = w0
    h = 0
    
    for _ in range(iterations):
        if Nesterov:
            grad = X.T @ ((X @ (params - beta * h)) - y) # учитываем 'импульс'

        else:
            grad = X.T @ ((X @ params) - y)
        h =  beta * h - (2./m) * learning_rate * grad
        
        if np.linalg.norm(h) < eps:
            return params
        params -= h

    return params

In [96]:
def RMSprop(X: np.array, y : np.array, w0: np.array, iterations: int, learning_rate=0.01, beta=0.9, eps=1e-3):
    '''Функция поиска минимума функции MSE в векторном виде с использованием метода Root Mean Squared propagation'''
    m = X.shape[0] # кол-во объектов
    if w0.shape[-1] != X.shape[-1] + 1:
        return 'Неверно задана матрица весов w0. Проверьте количество признаков'
    X = np.hstack((np.ones((m, 1)), X)) # добавляем константу для w0
    params = w0
    G = np.zeros_like(params)
    
    for _ in range(iterations):
        grad = X.T @ ((X @ params) - y)  
        G = beta * G + (1 - beta) * grad ** 2  # Сглаживание квадратов градиента
        update = (learning_rate / (np.sqrt(G) + eps)) * grad  # Обновление параметров

        if np.linalg.norm(update) < eps:  
            return params
        
        params -= update  
        
    return params

In [106]:
def Adam(X: np.array, y : np.array, w0: [np.array, list, tuple], iterations: int, learning_rate: float, beta_1: float, beta_2: float, eps=1e-3):
    '''Функция для поиска минимума функции MSE в векторном виде с использоваением метода Adam'''
    m = X.shape[0] # кол-во объектов
    if w0.shape[-1] != X.shape[-1] + 1:
        return 'Неверно задана матрица весов w0. Проверьте количество признаков'
    X = np.hstack((np.ones((m, 1)), X)) # добавляем константу для w0
    params = np.array(w0)
    h = np.zeros_like(params)
    G = np.zeros_like(params)
    
    for i in range(1, iterations+1):
        grad = X.T @ ((X @ params) - y) 
        h = (beta_1 * h + (1 - beta_1) * grad) / (1 - beta_1)
        G = (beta_2 * G + (1 - beta_2) * grad ** 2) / (1 - beta_2)  # Сглаживание квадратов градиента

        # Коррекция смещенния для компенсации нулевой инициализации
        h_t = h / (1 - beta_1 ** i)
        G_t = G / (1 - beta_2 ** i)
        update = (learning_rate / (np.sqrt(G_t) + eps)) * h_t # Обновление параметров

        if np.linalg.norm(update) < eps:  
            return params
        
        params -= update  
        
    return params